In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
import requests
import json
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!pip install folium
import folium 
!pip install OSGridConverter
from OSGridConverter import grid2latlong

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [2]:
london_source = "https://en.wikipedia.org/wiki/List_of_areas_of_London"

In [3]:
data  = requests.get(london_source).text
#creating a beautiful soup object
soup = BeautifulSoup(data,"html5lib")
tables = soup.find_all('table')
print(len(tables))

5


In [4]:
for index,table in enumerate(tables):
    if ("TQ465785" in str(table)):
        table_index = index
print(table_index)

1


In [5]:
table_data=pd.DataFrame(columns=["London Neighborhood", "London borough", "Post town", "Postcode", "Dial code", "OS grid ref"])

In [6]:
for row in tables[table_index].tbody.find_all("tr"):
    col = row.find_all("td")
    if (col != []):
        location = col[0].text
        borough = col[1].text
        post_town = col[2].text
        pc = col[3].text
        dial_code = col[4].text
        os_grid = col[5].text
        table_data = table_data.append({"London Neighborhood":location, "London borough":borough, "Post town":post_town, "Postcode":pc, "Dial code":dial_code, "OS grid ref":os_grid}, ignore_index=True)

table_data.head()

,London Neighborhood,London borough,Post town,Postcode,Dial code,OS grid ref
0,Abbey Wood,"Bexley, Greenwich [7]",LONDON,SE2,020,TQ465785\n
1,Acton,"Ealing, Hammersmith and Fulham[8]",LONDON,"W3, W4",020,TQ205805\n
2,Addington,Croydon[8],CROYDON,CR0,020,TQ375645\n
3,Addiscombe,Croydon[8],CROYDON,CR0,020,TQ345665\n
4,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14",020,TQ478728\n


In [7]:
table_data["London borough"]=table_data["London borough"].str.replace(r"\[.*\]","")
table_data["OS grid ref"] = table_data["OS grid ref"].replace(r'\n',' ', regex=True)
table_data.loc[53,'OS grid ref']='TQ478742'
table_data.loc[232,'OS grid ref']='TQ4453061645'
table_data

,London Neighborhood,London borough,Post town,Postcode,Dial code,OS grid ref
0,Abbey Wood,"Bexley, Greenwich",LONDON,SE2,020,TQ465785
1,Acton,"Ealing, Hammersmith and Fulham",LONDON,"W3, W4",020,TQ205805
2,Addington,Croydon,CROYDON,CR0,020,TQ375645
3,Addiscombe,Croydon,CROYDON,CR0,020,TQ345665
4,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14",020,TQ478728
5,Aldborough Hatch,Redbridge,ILFORD,IG2,020,TQ455895
6,Aldgate,City,LONDON,EC3,020,TQ334813
7,Aldwych,Westminster,LONDON,WC2,020,TQ307810
8,Alperton,Brent,WEMBLEY,HA0,020,TQ185835
9,Anerley,Bromley,LONDON,SE20,020,TQ345695


In [8]:
def get_coordinates(OS_Grid_ref):
    OS_Grid_ref=str(OS_Grid_ref)
    l=grid2latlong(OS_Grid_ref)
    lat=l.latitude
    long=l.longitude
    return pd.Series([lat,long])

In [9]:
coordinates = pd.DataFrame(table_data.apply(lambda x: get_coordinates(x["OS grid ref"]),axis=1))
coordinates.rename(columns ={0:'Latitude',1:'Longitude'},inplace = True)
coordinates.head()

,Latitude,Longitude
0,51.486484,0.109318
1,51.510591,-0.264585
2,51.362934,-0.025780
3,51.381625,-0.068126
4,51.434929,0.125663


In [10]:
table_data['Latitude']=coordinates['Latitude']
table_data['Longitude']=coordinates['Longitude']
table_data=table_data.drop(['Post town','Dial code','OS grid ref'], axis=1)
table_data.head()

,London Neighborhood,London borough,Postcode,Latitude,Longitude
0,Abbey Wood,"Bexley, Greenwich",SE2,51.486484,0.109318
1,Acton,"Ealing, Hammersmith and Fulham","W3, W4",51.510591,-0.264585
2,Addington,Croydon,CR0,51.362934,-0.025780
3,Addiscombe,Croydon,CR0,51.381625,-0.068126
4,Albany Park,Bexley,"DA5, DA14",51.434929,0.125663


In [11]:
CLIENT_ID = 'DUP4DI24U3MAYYFBGMWJLI52D1CTHO22OLOK32JLFLRUIWGR' # your Foursquare ID
CLIENT_SECRET = 'FHU5AUTSNXEJDQ0K5315HZBC4J1ITQKPSF2JIUNAO4JQCMKX' # your Foursquare Secret
VERSION='20180605'

In [12]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [16]:
def getNearbyVenues(names, lat, lng,radius,limit):
        url2 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            lat, 
            lng, 
            VERSION,
            QUERY,
            radius,
            limit)
        results = requests.get(url2).json()
        venues = results['response']['venues']
        dataframe = pd.json_normalize(venues)
        
        if len(dataframe.columns) == 0:
            return None
        
        else:
            filtered_columns = ['name', 'categories','location.distance','location.address','location.lat','location.lng','location.postalCode']
            temp_df = dataframe.reindex(columns = filtered_columns)
            #filter the category for each row
            temp_df['categories'] = temp_df.apply(get_category_type, axis=1)
            temp_df.insert(0, 'LondonNeighborhood', names)
            temp_df.insert(1,'NeighborhoodLatitude',lat)
            temp_df.insert(2,'NeighborhoodLongitude',lng)
            temp_df.columns = ['LondonNeighborhood', 
                             'NeighborhoodLatitude', 
                             'NeighborhoodLongitude', 
                             'VenueName',
                             'VenueCategory',
                             'VenueDistance',
                             'VenueAddress',
                             'VenueLat',
                             'VenueLng',
                             'VenuePostalCode'
                             ]
            return temp_df

In [17]:
QUERY='college university'
radius=1000
limit=10

london_venues=pd.DataFrame(columns=['LondonNeighborhood', 
                             'NeighborhoodLatitude', 
                             'NeighborhoodLongitude', 
                             'VenueName',
                             'VenueCategory',
                             'VenueDistance',
                             'VenueAddress',
                             'VenueLat',
                             'VenueLng',
                             'VenuePostalCode'
                             ])

for i in range(len(table_data)):
    print(table_data.loc[i,'London Neighborhood'])
    temp=getNearbyVenues(table_data.loc[i,'London Neighborhood'], table_data.loc[i,'Latitude'], table_data.loc[i,'Longitude'],radius,limit)
    london_venues=london_venues.append(temp)
    
london_venues.reset_index(inplace=True)
london_venues.drop('index',axis=1,inplace=True)
london_venues.head()

Abbey Wood
Acton


KeyError: 'venues'

In [ ]:
venue_cat=london_venues['VenueCategory'].unique()
df=pd.DataFrame(venue_cat, columns=['Categories'])
df

In [ ]:
look_for=['Adult Education Center',
          'College & University',
          'General College & University',
          'University',
          'Medical School',
          'Law School',  
          'College Residence Hall',
          'School',
          'High School',
          'Community College',
          'Music School',
          'Residential Building (Apartment / Condo)',
          'Language School',
          'Trade School',
          'Library',
          'Convention Center'
            ]
shortlist= london_venues[pd.DataFrame(london_venues['VenueCategory'].tolist()).isin(look_for).any(1).values]
print(shortlist.shape)
shortlist.head(3)

In [ ]:
Neighborhood_target=shortlist.groupby('LondonNeighborhood').mean()
Neighborhood_target=Neighborhood_target.drop(['VenueLat','VenueLng'],axis=1).reset_index()

In [ ]:
Neighborhood_target.head()

In [ ]:
address = 'London, UK'

geolocator = Nominatim(user_agent="on_search")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London are {}, {}.'.format(latitude, longitude))

In [ ]:
# create map of UK using latitude and longitude values

map_UK = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, label in zip(Neighborhood_target['NeighborhoodLatitude'], Neighborhood_target['NeighborhoodLongitude'], Neighborhood_target['LondonNeighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_UK)  
    
map_UK

In [ ]:
# one hot encoding
london_onehot = pd.get_dummies(shortlist[['VenueCategory']], prefix="", prefix_sep="")
london_onehot
# add neighborhood column back to dataframe
london_onehot.insert(0, "LondonNeighborhood", shortlist["LondonNeighborhood"], True)
london_onehot.insert(1, "NeighborhoodLatitude", shortlist["NeighborhoodLatitude"], True)
london_onehot.insert(2, "NeighborhoodLongitude", shortlist["NeighborhoodLongitude"], True)

london_onehot.head()

In [ ]:
london_grouped = london_onehot.groupby('LondonNeighborhood').mean().reset_index()
london_grouped.head()

In [ ]:
# set number of clusters
kclusters = 3

london_grouped_clustering = london_grouped.drop('LondonNeighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

In [ ]:
london_grouped.insert(0, 'Cluster Labels', kmeans.labels_)

In [ ]:
london_grouped.head(10)

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_grouped['NeighborhoodLatitude'], london_grouped['NeighborhoodLongitude'], london_grouped['LondonNeighborhood'], london_grouped['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
cluster_data=london_grouped.groupby('Cluster Labels').mean()


In [ ]:
cluster_t=cluster_data.transpose()

In [ ]:
cluster_t=cluster_t.drop(['NeighborhoodLatitude', 'NeighborhoodLongitude'],axis=0)
cluster_t

In [ ]:
cluster_t.rename_axis('Venue Category', axis=1)
cluster_t.columns=[ 'Cluster 0', 'Cluster 1', 'Cluster 2' ]
cluster_t

In [ ]:
cluster_0=cluster_t['Cluster 0']
cluster_1=cluster_t['Cluster 1']
cluster_2=cluster_t['Cluster 2']

In [ ]:
%matplotlib inline 

import matplotlib as mpl
import matplotlib.pyplot as plt

cluster_0.plot(kind='barh', figsize=(8, 4))

plt.xlabel('Venue Category') # add to x-label to the plot
plt.ylabel('Density') # add y-label to the plot
plt.title('Venue Categories for Cluster 0') # add title to the plot

plt.show()

In [ ]:
cluster_1.plot(kind='barh', figsize=(8, 4))

plt.xlabel('Venue Category') # add to x-label to the plot
plt.ylabel('Density') # add y-label to the plot
plt.title('Venue Categories for Cluster 1') # add title to the plot

plt.show()

In [ ]:
cluster_2.plot(kind='barh', figsize=(8, 4))

plt.xlabel('Venue Category') # add to x-label to the plot
plt.ylabel('Density') # add y-label to the plot
plt.title('Venue Categories for Cluster 0') # add title to the plot

plt.show()